In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LassoCV

In [8]:
df=pd.read_csv('Hitters.csv')

In [6]:
df.head()

,Unnamed: 0,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,...,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,-Andy Allanson,293,66,1,30,29,14,1,293,66,...,30,29,14,A,E,446,33,20,NaN,A
1,-Alan Ashby,315,81,7,24,38,39,14,3449,835,...,321,414,375,N,W,632,43,10,475.0,N
2,-Alvin Davis,479,130,18,66,72,76,3,1624,457,...,224,266,263,A,W,880,82,14,480.0,A
3,-Andre Dawson,496,141,20,65,78,37,11,5628,1575,...,828,838,354,N,E,200,11,3,500.0,N
4,-Andres Galarraga,321,87,10,39,42,30,2,396,101,...,48,46,33,N,E,805,40,4,91.5,N


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  322 non-null    object 
 1   AtBat       322 non-null    int64  
 2   Hits        322 non-null    int64  
 3   HmRun       322 non-null    int64  
 4   Runs        322 non-null    int64  
 5   RBI         322 non-null    int64  
 6   Walks       322 non-null    int64  
 7   Years       322 non-null    int64  
 8   CAtBat      322 non-null    int64  
 9   CHits       322 non-null    int64  
 10  CHmRun      322 non-null    int64  
 11  CRuns       322 non-null    int64  
 12  CRBI        322 non-null    int64  
 13  CWalks      322 non-null    int64  
 14  League      322 non-null    object 
 15  Division    322 non-null    object 
 16  PutOuts     322 non-null    int64  
 17  Assists     322 non-null    int64  
 18  Errors      322 non-null    int64  
 19  Salary      263 non-null    f

In [9]:
df['Salary'].fillna(df['Salary'].median(skipna=True),inplace=True)
df.isna().sum()

Unnamed: 0    0
AtBat         0
Hits          0
HmRun         0
Runs          0
RBI           0
Walks         0
Years         0
CAtBat        0
CHits         0
CHmRun        0
CRuns         0
CRBI          0
CWalks        0
League        0
Division      0
PutOuts       0
Assists       0
Errors        0
Salary        0
NewLeague     0
dtype: int64

In [12]:
#Dummy variables
dms = pd.get_dummies(df[['League','Division','NewLeague']],drop_first = True)

In [15]:
y = df['Salary']
x_ = df.drop(['Unnamed: 0','Salary','League','Division','NewLeague'],axis  = 1).astype('float64')
X = pd.concat([x_, dms[['League_N','Division_W','NewLeague_N']]],axis=1)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [18]:
lasso_model = Lasso().fit(X_train,y_train)
lasso_model.intercept_

C:\Apps\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.923e+06, tolerance: 3.853e+03
  model = cd_fast.enet_coordinate_descent(


342.8733925858776

In [19]:
lasso_model.coef_

array([-1.98558949e+00,  5.50494749e+00,  4.79612807e+00,  1.02123896e-01,
       -8.11521080e-01,  4.87004116e+00, -9.97808288e+00, -2.19391227e-01,
        6.16237616e-01,  9.03214960e-03,  8.73990383e-01,  7.84172593e-01,
       -8.13423037e-01,  1.83989460e-01,  4.04846687e-01, -4.08650952e+00,
        2.67092023e+01, -1.11463261e+02, -0.00000000e+00])

In [20]:
y_pred = lasso_model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

345.6190692407428

In [21]:
r2_score(y_test, y_pred)

0.3657513009571691

In [23]:
#Optimization using Cross Validation
lasso_cv_model = LassoCV(alphas=np.random.randint(0, 1000, 100), cv=10, max_iter=10000, n_jobs=-1).fit(X_train,y_train)

In [24]:
lasso_cv_model.alpha_

15

In [25]:
lasso_tuned = Lasso().set_params(alpha = 14).fit(X_train, y_train)
y_pred_tuned = lasso_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred_tuned))

C:\Apps\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.397e+06, tolerance: 3.853e+03
  model = cd_fast.enet_coordinate_descent(


346.70839160783527

In [26]:
#Values of the Optimized Lasso Regression
pd.Series(lasso_tuned.coef_,index=X_train.columns)

AtBat          -1.918086
Hits            5.126820
HmRun           2.443879
Runs            0.295663
RBI            -0.000000
Walks           4.821499
Years          -3.879546
CAtBat         -0.263819
CHits           0.734330
CHmRun          0.132193
CRuns           0.864787
CRBI            0.735014
CWalks         -0.786518
PutOuts         0.178432
Assists         0.382404
Errors         -2.613340
League_N        0.000000
Division_W    -55.486277
NewLeague_N     0.000000
dtype: float64